In [17]:
from rlcms.sampling import strat_sample_from_reference, strat_sample, split_train_test
import ee
ee.Initialize(project='sig-ee-cloud')
img = ee.Image("projects/sig-ee-cloud/assets/rlcmsTests/composite_S2_sesheke_2021_22")
lc = ee.Image("projects/sig-ee-cloud/assets/rlcmsTests/LandCover_Sesheke")

pts = strat_sample(lc,'LANDCOVER',lc.geometry(),10,90210,50,None,None)
polys = pts.map(lambda p: p.buffer(50).bounds())
print(f"Dummy reference points count: {pts.size().getInfo()}")

Dummy reference points count: 400


In [ ]:
# can extract image band information using reference points or polygons
samples_from_pts = strat_sample_from_reference(img,pts,'LANDCOVER',10,None,123,[1,2,3],[10,10,10])
samples_from_polys = strat_sample_from_reference(img,polys,'LANDCOVER',10,None,456,[1,2,3],[10,10,10])

# get class breakdowns
print(f"Breakdown (samples from pts): {samples_from_pts.aggregate_histogram('LANDCOVER').getInfo()}")
print(f"Breakdown (samples from polys): {samples_from_polys.aggregate_histogram('LANDCOVER').getInfo()}")


In [16]:
import geemap
Map = geemap.Map()

Map.addLayer(img.select(0,1,2),{'min':0,'max':1500},'Composite')
Map.addLayer(pts,{},'pts')
Map.addLayer(polys,{},'polys')
Map.addLayer(samples_from_pts,{'color':'red'},'samples_from_reference(pts)')
Map.addLayer(samples_from_polys,{'color':'blue'},'samples_from_reference(polys)')
Map.centerObject(img,12)
Map

Map(center=[-17.479823520857828, 24.29799114976096], controls=(WidgetControl(options=['position', 'transparent…

In [4]:
# split into train/test
train,test = split_train_test(samples_from_polys,101010)
print(f"train point count: {train.size().getInfo()}")
print(f"test point count: {test.size().getInfo()}")

train point count: 13
test point count: 4
['LANDCOVER', 'aspect', 'change_abs', 'change_norm', 'eastness', 'elevation', 'max_extent', 'northness', 'occurrence', 'seasonality', 'slope', 'swir1_amplitude', 'swir1_phase', 'system:index', 't0_EVI_median', 't0_IBI_median', 't0_blue_median', 't0_green_median', 't0_nir_median', 't0_red_median', 't0_swir1_median', 't0_swir2_median', 't1_EVI_median', 't1_IBI_median', 't1_blue_median', 't1_green_median', 't1_nir_median', 't1_red_median', 't1_swir1_median', 't1_swir2_median', 'transition']


In [8]:
Map = geemap.Map()

Map.addLayer(img.select(0,1,2),{'min':0,'max':1500},'Composite')
Map.addLayer(samples_from_polys,{'color':'grey'},'samples_from_reference(polys)')
Map.addLayer(train,{'color':'red'},'training pts')
Map.addLayer(test,{'color':'blue'},'testing pts')
Map.centerObject(img,14)
Map

Map(center=[-17.479823520857828, 24.29799114976096], controls=(WidgetControl(options=['position', 'transparent…

In [19]:
# export points to Asset or Drive
from rlcms.utils import exportTableToAsset,exportTableToDrive

exportTableToAsset(train,
                    'exportTrainingPoints-Asset',
                    'projects/sig-ee-cloud/assets/rlcmsTests/demoTrainPtsSesheke')
exportTableToDrive(train,
                    'exportTrainingPoints-Drive',
                    'rlcmsTests',
                    'demoTrainPtsSesheke',
                    None)

Export started (Asset): projects/sig-ee-cloud/assets/rlcmsTests/demoTrainPtsSesheke
Export started (Drive): rlcmsTests/demoTrainPtsSesheke
